# Using Xtract-SDK to extract metadata from files 

In [1]:
from xtract_sdk.client import XtractClient
# print(f"You are using xtract_sdk version: {xtract_sdk.__version__}")  # TODO: get this working. 

source_ep_path_1 = "/MDF/mdf_connect/prod/data/_test_einstein_9vpflvd_v1.1"
source_ep_id_1 = "e38ee745-6d04-11e5-ba46-22000b92c6ec"  # MDF at Petrel 
dest_ep_id_1 = "af7bda53-6d04-11e5-ba46-22000b92c6ec"  # where they will be extracted

# Globus endpoint whatand file path at which we want to store metadata documents
mdata_ep_id = "5113667a-10b4-11ea-8a67-0e35e66293c2"  
mdata_path = "/home/ubuntu/tskluzac"

# FuncX endpoint at which we want the metadata extraction to occur. Does NOT have to be same endpoint as the data.
funcx_ep_id = "b08b8612-cd87-47f4-b2ff-8c69c2c03d53"

# Grouping strategy
grouper = "file_is_group"

xc = XtractClient()


In [ ]:
print(auths)
fx_scope = 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/all'
headers = {'Authorization': f"Bearer {auths['petrel']}", 'Transfer': auths['transfer'], 'FuncX': auths[fx_scope], 'Petrel': auths['petrel']}
fx_headers = {'Authorization': f"Bearer {auths[fx_scope].access_token}",
             'Search': auths['search'].authorizer.access_token,
             'Openid': auths['openid'].access_token}
print("\n" + str(fx_headers))

## Step 2: Crawl
Crawling, behind the scenes, will scan a Globus directory breadth-first (using globus_ls), first extracting physical metadata such as path, size, and extension. Next, since the *grouper* we selected is 'matio', the crawler will execute matio's `get_groups_by_postfix()` function on all file names in a directory in order to return groups for each of matio's parsers (besides *generic* and *noop*). 

The crawl will run as a non-blocking thread, and return a crawl_id that will be used extensively to track progress of our metadata extraction workflow.

In [ ]:
# TODO: Adjust this to the Google Drive model!!!

crawl_url = f'{eb_url}/crawl'
print(f"Crawl URL is : {crawl_url}")

first_ep_dict = {
    'repo_type': 'GLOBUS',
    'eid': source_ep_id,
    'dir_paths': [source_ep_path_1],# , source_ep_path_2],
    'grouper': grouper, 
    # 'prefetch_list':
    #     [{'dest_ep_id': dest_ep_id, 'path': '/project2/chard/skluzacek/data_to_process'}]
}

tokens = {'Transfer': auths['transfer'].authorizer.access_token, 
          'Authorization': f"Bearer {auths['petrel'].access_token}", 
          'FuncX': auths[fx_scope].access_token}
print(tokens)

crawl_req = requests.post(crawl_url, json={'endpoints': [first_ep_dict], 'tokens': tokens})
print(crawl_req.content)
crawl_id = json.loads(crawl_req.content)['crawl_id']
print(f"Crawl ID: {crawl_id}")


We can get crawl status, seeing how many groups have been identified in the crawl. 

Note that measuring the total files yet to crawl is impossible, as the BFS may not have discovered all files yet, and Globus does not yet have a file counting feature for all directories and subdirectories. I.e., we know when we're done, but we don't know until we get there. 

In [ ]:
# crawl_id = '506c72c4-36dc-461f-8c69-4ed591f75b0a'
# TODO: update the crawl status to query the DB (might require occasionally updating db)
crawl_status = requests.get(f'{eb_url}/get_crawl_status', json={'crawl_id': crawl_id})
print(crawl_status)
crawl_content = json.loads(crawl_status.content)
print(f"Crawl Status: {crawl_content}")

In [ ]:
import time
while True:
    req = requests.get(f'{eb_url}/fetch_crawl_mdata', json={'crawl_id': crawl_id, 'n': 100})
    print(req.content)
    time.sleep(1)

In [ ]:
# crawl_id
# fetch_mdata = requests.get(f'{eb_url}/fetch_crawl_mdata', json={'crawl_id': crawl_id, 'n': 2})
# print(fetch_mdata.content)
# # fetch_content = json.loads(fetch_mdata.content)
# print(f"Crawl Status: {crawl_content}")

# tokens['Bearer']
print(f"Tokens: {tokens}")



# HERE WE WILL TEST CONFIGURING OUR ENDPOINT. 
config_status = requests.post(f"{eb_url}/configure_ep/{funcx_ep_id}", json={'headers': fx_headers, 
                                                                            'timeout': 25, 
                                                                            'ep_name': 'tyler_test_ep_2', 
                                                                            'globus_eid': '12345', 
                                                                            'xtract_path':'/Users/tylerskluzacek/.xtract',
                                                                            'local_download_path': 'foobar',
                                                                            'local_mdata_path': '/Users/tylerskluzacek/Desktop/metadata'
                                                                            })
config_content = json.loads(config_status.content)
print(f"Returned: {config_content}")


## Step 3: Xtract

Next we launch a non-blocking metadata extraction workflow that will automatically find all groups generated from our crawl_id, ship parsers to our endpoint as funcX, transfer the file (if necessary), and extract/send back metadata to the central Xtract service. This will just run constantly until the crawl is done and there are crawled groups left to extract. 

In [ ]:
# crawl_id = "246d0edf-2641-46f7-9700-3cc5a49c4890"
xtract = requests.post(f'{eb_url}/extract', json={'crawl_id': crawl_id,
                                                  'repo_type': "HTTPS",
                                                  'headers': json.dumps(headers),
                                                  'funcx_eid': funcx_ep_id, 
                                                  'source_eid': source_ep_id,
                                                  'dest_eid': dest_ep_id,
                                                  'mdata_store_path': mdata_path, 
                                                  'data_prefetch_path': data_prefetch_path, 
                                                  'prefetch_remote': True})
print(f"Xtract response (should be 200): {xtract}")

In [ ]:
xtract_status = requests.get(f'{eb_url}/get_extract_status', json={'crawl_id': crawl_id})
xtract_content = json.loads(xtract_status.content)
print(f"Xtract Status: {xtract_content}")

## Step 4: Access / Flush

We might want to flush all new metadata blobs to a separate Globus endpoint. Here we initialize a results poller that creates a file of each metadata attribute to a folder at this path: `<mdata_path>/<crawl_id>/<group_id>`

In [ ]:
import time 

while True: 
    poller = requests.post(f'{eb_url}/fetch_crawl_mdata', json={'crawl_id': crawl_id, 'Transfer': transfer_token, 'n': 100})
    print(f'Flush Status: {poller}')
    time.sleep(5)